In [5]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn import cross_validation
%matplotlib inline

ImportError: cannot import name 'cross_validation' from 'sklearn' (/home/szantamano/.local/lib/python3.8/site-packages/sklearn/__init__.py)

### 5.3.1 The Validation Set Approach
We explore the use of the validation set approach in order to estimate the
test error rates that result from fitting various linear models on the Auto
data set.
Before we begin, we use the set.seed() function in order to set a seed for seed
R’s random number generator, so that the reader of this book will obtain
precisely the same results as those shown below. It is generally a good idea
to set a random seed when performing an analysis such as cross-validation
that contains an element of randomness, so that the results obtained can
be reproduced precisely at a later time.
We begin by using the sample() function to split the set of observations sample() into two halves, by selecting a random subset of 196 observations out of
the original 392 observations. We refer to these observations as the training
set.

In [3]:
# Load dataset
# We later want to fit using horsepower, so remove its rows with NA
auto = pd.read_csv('Data/Auto.csv', na_values='?')
auto = auto.dropna()

X = np.array(auto["horsepower"]).reshape(-1, 1)
Y = np.array(auto["mpg"]).reshape(-1, 1)

# Set the random seed
np.random.seed(1)

# Train and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=0)

We then fit a linear regression using only the observations corresponding to the training set, and calcualte the MSE on the test set.

In [4]:
model = LinearRegression().fit(X_train, y_train)
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
display(f"The MSE for linear regression is {mse}")

'The MSE for linear regression is 23.61661706966988'

Therefore, the estimated test MSE for the linear regression fit is 23.61. We can also estimate the test error for the quadratic and cubic regressions.

In [5]:
# Quadratic regression
poly2 = PolynomialFeatures(2)

X_v2= poly2.fit_transform(X)

X_train_v2, X_test_v2, y_train_v2, y_test_v2 = train_test_split(X_v2, Y, test_size=0.5, random_state=0)

model = LinearRegression().fit(X_train_v2, y_train_v2)
predictions_v2 = model.predict(X_test_v2)

mse_v2 = mean_squared_error(y_test_v2, predictions_v2)
display(f"The MSE for quadratic regression is {mse_v2}")

# Cubic regression

poly3 = PolynomialFeatures(3)

X_v3= poly3.fit_transform(X)

X_train_v3, X_test_v3, y_train_v3, y_test_v3 = train_test_split(X_v3, Y, test_size=0.5, random_state=0)

model = LinearRegression().fit(X_train_v3, y_train_v3)
predictions_v3 = model.predict(X_test_v3)

mse_v3 = mean_squared_error(y_test_v3, predictions_v3)
f"The MSE for cubic regression is {mse_v3}"


'The MSE for quadratic regression is 18.763031346897684'

'The MSE for cubic regression is 18.79694163262019'

These error rates are 18.76 and 18.79, respectively. If we choose a different
training set instead, then we will obtain somewhat different errors on the
validation set.

In [6]:
# Linear regression
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=3)
model = LinearRegression().fit(X_train, y_train)
predictions = model.predict(X_test)
mse = mean_squared_error(y_test, predictions)
display(f"The MSE for linear regression is {mse}")
# Quadratic regression
poly2 = PolynomialFeatures(2, include_bias = False)
X_v2= poly2.fit_transform(X)
X_train_v2, X_test_v2, y_train_v2, y_test_v2 = train_test_split(X_v2, Y, test_size=0.5, random_state=3)
model = LinearRegression().fit(X_train_v2, y_train_v2)
predictions_v2 = model.predict(X_test_v2)
mse_v2 = mean_squared_error(y_test_v2, predictions_v2)
display(f"The MSE for quadratic regression is {mse_v2}")
# Cubic regression
poly3 = PolynomialFeatures(3, include_bias = False)
X_v3= poly3.fit_transform(X)
X_train_v3, X_test_v3, y_train_v3, y_test_v3 = train_test_split(X_v3, Y, test_size=0.5, random_state=3)
model = LinearRegression().fit(X_train_v3, y_train_v3)
predictions_v3 = model.predict(X_test_v3)
mse_v3 = mean_squared_error(y_test_v3, predictions_v3)
f"The MSE for cubic regression is {mse_v3}"


'The MSE for linear regression is 20.755407959228602'

'The MSE for quadratic regression is 16.945106759516108'

'The MSE for cubic regression is 16.974378328026475'

Using this split of the observations into a training set and a validation
set, we find that the validation set error rates for the models with linear,
quadratic, and cubic terms are 20.75, 16.94, and 16.97, respectively.
These results are consistent with our previous findings: a model that
predicts mpg using a quadratic function of horsepower performs better than
a model that involves only a linear function of horsepower, and there is
little evidence in favor of a model that uses a cubic function of horsepower.

### 5.3.2 Leave-One-Out Cross-Validation
The LOOCV estimate can be automatically computed for any generalized
linear model using sklearn.

In [7]:
def cv_poly_reg(degree, X, Y):
    poly = PolynomialFeatures(degree = degree, include_bias = False)
    X = poly.fit_transform(X)
    model = LinearRegression()
    scores = cross_val_score(model, X, Y, cv= X.shape[0], scoring = "neg_mean_squared_error" )
    mse = np.mean(scores)
    display(f"The MSE for degree {degree} regression is {mse}")

for degree in range(1,6):
    cv_poly_reg(degree, X, Y)

'The MSE for degree 1 regression is -24.231513517929226'

'The MSE for degree 2 regression is -19.24821312448967'

'The MSE for degree 3 regression is -19.334984064029197'

'The MSE for degree 4 regression is -19.424430310319195'

'The MSE for degree 5 regression is -19.03321350576735'

we see a sharp drop in the estimated test MSE between
the linear and quadratic fits, but then no clear improvement from using
higher-order polynomials

### 5.3.3 k-Fold Cross-Validation

The same functions can also be used to implement k-fold CV. Below we use k = 10, a common choice for k, on the Auto data set. We simply copy the lines of code above using 10 folds instead of n folds in the cross-validation to be explicit.

In [8]:
def cv_poly_reg(degree, X, Y):
    poly = PolynomialFeatures(degree = degree, include_bias = False)
    X = poly.fit_transform(X)
    model = LinearRegression()
    scores = cross_val_score(model, X, Y, cv= 10, scoring = "neg_mean_squared_error" )
    mse = np.mean(scores)
    display(f"The MSE for degree {degree} regression is {mse}")

for degree in range(1,10):
    cv_poly_reg(degree, X, Y)

'The MSE for degree 1 regression is -27.439933652339867'

'The MSE for degree 2 regression is -21.235840055802228'

'The MSE for degree 3 regression is -21.33660618322802'

'The MSE for degree 4 regression is -21.353886981979993'

'The MSE for degree 5 regression is -20.905640019587945'

'The MSE for degree 6 regression is -20.77902044138328'

'The MSE for degree 7 regression is -20.98683005802863'

'The MSE for degree 8 regression is -21.077322981225386'

'The MSE for degree 9 regression is -21.03788551256914'

We still see little evidence that using
cubic or higher-order polynomial terms leads to lower test error than simply
using a quadratic fit.

### 5.3.4 The Bootstrap

We illustrate the use of the bootstrap in the simple example of Section 5.2,
as well as on an example involving estimating the accuracy of the linear
regression model on the Auto data set.

## Estimating the Accuracy of a Statistic of Interest
One of the great advantages of the bootstrap approach is that it can be
applied in almost all situations. No complicated mathematical calculations
are required. Performing a bootstrap analysis entails only two steps.
First, we must create a function that computes the statistic of interest.
Second, perform the bootstrap by repeatedly sampling observations from the data
set with replacement.
The Portfolio data set in the ISLR package is described in Section 5.2.
To illustrate the use of the bootstrap on this data, we must first create
a function, alpha.fn(), which takes as input the (X, Y ) data as well as
a vector indicating which observations should be used to estimate α. The
function then outputs the estimate for α based on the selected observations.


In [8]:
# load the portfolio data set
portfolio = pd.read_csv('Data/Portfolio.csv', index_col=0)
print('\n',len(portfolio),'Rows')
portfolio.head()


 100 Rows


,X,Y
1,-0.895251,-0.234924
2,-1.562454,-0.885176
3,-0.417090,0.271888
4,1.044356,-0.734198
5,-0.315568,0.841983


In [10]:


def estimate_alpha(X, y, index):
    X= X[index]
    y = y[index]
    return((np.var(y)-np.cov(X,y)[0,1])/(np.var(X)+np.var(y)-2*np.cov(X, y)[0,1]))


This function returns, or outputs, an estimate for α based on applying
(5.7) to the observations indexed by the argument index. For instance, the
following command estimates α using all 100 observations.

In [11]:
estimate_alpha(portfolio.X, portfolio.Y, range(1,101))

0.5766511516104116

The next command uses the sample() function to randomly select 100 observations from the range 1 to 100, with replacement. This is equivalent to constructing a new bootstrap data set and recomputing $\hat\alpha$ based on the new data set.

In [12]:
np.random.seed(0)
index = np.random.choice(range(1,101), 100, replace=True)
estimate_alpha(portfolio.X, portfolio.Y, index)

0.560336658007497

We can implement a bootstrap analysis by performing this command many
times, recording all of the corresponding estimates for α, and computing the resulting standard deviation.

In [20]:
def do_bootstrap(portfolio, times):
    results = []
    for t in range(times):
       index = np.random.choice(range(1,101), 100, replace=True)
       result = estimate_alpha(portfolio.X, portfolio.Y, index)
       results.append(result)
    
    results = np.array(results)
    display(f"Our mean estimate is {np.mean(results)}")
    display(f"Standard deviation for our estimate is:{np.std(results)}")
    return np.std(results)

do_bootstrap(portfolio, 1000)

'Our mean estimate is 0.5793949782690855'

'Standard deviation for our estimate is:0.09415316225687384'

0.09415316225687384